# 💱 Real USD/KES Exchange Rate Modeling & Prediction

**Central Bank of Kenya Foreign Exchange Analysis**  
*Advanced FX Rate Forecasting using Real Market Data*

---

## Analysis Framework
- **Real Data Integration**: Actual CBK exchange rate data and market indicators
- **Advanced Modeling**: ARIMA, LSTM, and Ensemble forecasting methods
- **Market Microstructure**: Bid-ask spreads, volatility clustering, and intervention detection
- **Policy Impact**: Central bank intervention analysis and policy effects
- **Risk Management**: VaR estimation and stress testing scenarios

In [1]:
# Real FX Rate Modeling with Actual CBK Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Advanced Analytics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Time Series Analysis
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Neural Networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Financial Analysis
from arch import arch_model
import scipy.stats as stats

print("🚀 Real FX Modeling Engine Initialized")
print("💱 Loading actual CBK exchange rate data...")

class RealFXModeler:
    """Advanced FX rate modeling using actual CBK market data"""
    
    def __init__(self):
        self.data = {}
        self.models = {}
        self.forecasts = {}
        self.accuracy_scores = {}
        
        # FX market colors
        self.colors = {
            'usd_kes': '#1f77b4',
            'volatility': '#ff7f0e', 
            'volume': '#2ca02c',
            'spread': '#d62728',
            'trend': '#9467bd'
        }
        
    def load_real_fx_data(self):
        """Load actual CBK exchange rate datasets"""
        try:
            # Load exchange rate data
            fx_monthly_end = pd.read_csv('../data/raw/Monthly exchange rate (end period).csv')
            fx_monthly_avg = pd.read_csv('../data/raw/Monthly Exchange rate (period average).csv')
            trade_weighted = pd.read_csv('../data/raw/TRADE WEIGHTED AVERAGE INDICATIVE RATES.csv')
            
            print(f"✅ Loaded FX datasets:")
            print(f"   📊 Monthly End Period: {len(fx_monthly_end)} records")
            print(f"   📊 Monthly Average: {len(fx_monthly_avg)} records") 
            print(f"   📊 Trade Weighted Rates: {len(trade_weighted)} records")
            
            # Store raw data
            self.data['fx_end'] = fx_monthly_end
            self.data['fx_avg'] = fx_monthly_avg
            self.data['trade_weighted'] = trade_weighted
            
            # Display data structure
            print("\\n📋 FX End Period Data Structure:")
            print(fx_monthly_end.head())
            print("\\nColumns:", fx_monthly_end.columns.tolist())
            
            return fx_monthly_end, fx_monthly_avg, trade_weighted
            
        except Exception as e:
            print(f"❌ Error loading FX data: {e}")
            return None, None, None
    
    def preprocess_fx_data(self, fx_df):
        """Process FX data for analysis"""
        if fx_df is None:
            return None
            
        try:
            # Clean the data - the real structure has Year, Month, USD rate in specific columns
            df = fx_df.copy()
            
            # Skip the header row and use proper columns
            if len(df) > 1:
                # Use the second row onwards (first row is header info)
                df = df.iloc[1:].copy()
                
                # Check if we have Year, Month, and USD columns
                if len(df.columns) >= 3:
                    # Create proper column mapping
                    df.columns = ['Year', 'Month', 'USD_Rate'] + list(df.columns[3:])
                    
                    # Convert Year and Month to datetime
                    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
                    df['Month'] = pd.to_numeric(df['Month'], errors='coerce')
                    
                    # Create date column
                    df['date'] = pd.to_datetime(df[['Year', 'Month']].assign(day=1), errors='coerce')
                    
                    # Convert USD rate to numeric
                    df['usd_kes_rate'] = pd.to_numeric(df['USD_Rate'], errors='coerce')
                    
                    # Remove rows with missing critical data
                    df = df.dropna(subset=['date', 'usd_kes_rate'])
                    
                    if len(df) > 0:
                        # Sort by date
                        df = df.sort_values('date').reset_index(drop=True)
                        
                        # Calculate additional metrics
                        df['rate_change'] = df['usd_kes_rate'].diff()
                        df['rate_pct_change'] = df['usd_kes_rate'].pct_change()
                        df['volatility'] = df['rate_pct_change'].rolling(window=6).std()
                        
                        print(f"✅ Processed FX data: {len(df)} valid records")
                        print(f"📅 Date range: {df['date'].min()} to {df['date'].max()}")
                        print(f"💱 Rate range: {df['usd_kes_rate'].min():.2f} - {df['usd_kes_rate'].max():.2f}")
                        
                        return df
                    else:
                        print("❌ No valid data after processing")
                        return None
                else:
                    print("❌ Insufficient columns in FX data")
                    return None
            else:
                print("❌ Insufficient rows in FX data")
                return None
                
        except Exception as e:
            print(f"❌ Error processing FX data: {str(e)}")
            return None
    
    def analyze_fx_patterns(self, fx_df):
        """Comprehensive FX pattern analysis"""
        
        print("🔍 Conducting comprehensive FX analysis...")
        
        # Basic statistics
        print("\\n📊 FX Rate Statistics:")
        print(fx_df['usd_kes_rate'].describe())
        
        # Stationarity tests
        print("\\n🧪 Stationarity Tests:")
        
        # ADF test
        adf_result = adfuller(fx_df['usd_kes_rate'].dropna())
        print(f"ADF Test - p-value: {adf_result[1]:.4f}")
        print(f"ADF Test - Stationary: {'Yes' if adf_result[1] < 0.05 else 'No'}")
        
        # KPSS test
        kpss_result = kpss(fx_df['usd_kes_rate'].dropna())
        print(f"KPSS Test - p-value: {kpss_result[1]:.4f}")
        print(f"KPSS Test - Stationary: {'Yes' if kpss_result[1] > 0.05 else 'No'}")
        
        # Volatility analysis
        print("\\n📈 Volatility Analysis:")
        avg_volatility = fx_df['volatility'].mean()
        max_volatility = fx_df['volatility'].max()
        print(f"Average Volatility: {avg_volatility:.4f}")
        print(f"Maximum Volatility: {max_volatility:.4f}")
        
        # Trend analysis
        recent_trend = fx_df['trend'].tail(6).mean()
        print(f"Recent Trend (6M): {recent_trend:.4f}")
        print(f"Trend Direction: {'Appreciation' if recent_trend < 0 else 'Depreciation'}")
        
        return {
            'adf_pvalue': adf_result[1],
            'kpss_pvalue': kpss_result[1],
            'avg_volatility': avg_volatility,
            'recent_trend': recent_trend
        }
    
    def create_fx_visualization(self, fx_df):
        """Create comprehensive FX visualization dashboard"""
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('💱 USD/KES Exchange Rate', '📊 Rate Returns & Volatility',
                          '📈 Moving Averages', '🎯 Momentum Indicators'),
            specs=[[{"secondary_y": False}, {"secondary_y": True}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )
        
        # Main FX rate
        fig.add_trace(
            go.Scatter(
                x=fx_df['date'],
                y=fx_df['usd_kes_rate'],
                mode='lines',
                name='USD/KES Rate',
                line=dict(color=self.colors['usd_kes'], width=2)
            ),
            row=1, col=1
        )
        
        # Returns and volatility
        fig.add_trace(
            go.Scatter(
                x=fx_df['date'],
                y=fx_df['rate_return'] * 100,
                mode='lines',
                name='Monthly Returns (%)',
                line=dict(color=self.colors['trend'], width=1)
            ),
            row=1, col=2
        )
        
        # Add volatility on secondary y-axis
        fig.add_trace(
            go.Scatter(
                x=fx_df['date'],
                y=fx_df['volatility'],
                mode='lines',
                name='Volatility',
                line=dict(color=self.colors['volatility'], width=2, dash='dash'),
                yaxis='y2'
            ),
            row=1, col=2
        )
        
        # Moving averages
        for window, color in zip([3, 6, 12], ['#1f77b4', '#ff7f0e', '#2ca02c']):
            fig.add_trace(
                go.Scatter(
                    x=fx_df['date'],
                    y=fx_df[f'ma_{window}'],
                    mode='lines',
                    name=f'{window}M MA',
                    line=dict(color=color, width=1.5)
                ),
                row=2, col=1
            )
        
        # Momentum indicators
        fig.add_trace(
            go.Scatter(
                x=fx_df['date'],
                y=fx_df['momentum_3m'] * 100,
                mode='lines',
                name='3M Momentum (%)',
                line=dict(color=self.colors['trend'], width=2)
            ),
            row=2, col=2
        )
        
        fig.add_trace(
            go.Scatter(
                x=fx_df['date'],
                y=fx_df['momentum_12m'] * 100,
                mode='lines',
                name='12M Momentum (%)',
                line=dict(color=self.colors['spread'], width=2)
            ),
            row=2, col=2
        )
        
        # Add zero line for momentum
        fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=2)
        
        fig.update_layout(
            title="💱 USD/KES Exchange Rate - Comprehensive Analysis",
            template='plotly_dark',
            height=800,
            font=dict(color='white', size=11),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )
        
        # Update y-axes labels
        fig.update_yaxes(title_text="Exchange Rate", row=1, col=1)
        fig.update_yaxes(title_text="Returns (%)", row=1, col=2)
        fig.update_yaxes(title_text="Volatility", secondary_y=True, row=1, col=2)
        fig.update_yaxes(title_text="Exchange Rate", row=2, col=1)
        fig.update_yaxes(title_text="Momentum (%)", row=2, col=2)
        
        return fig

# Initialize the Real FX Modeler
fx_modeler = RealFXModeler()

print("\\n" + "="*60)
print("💱 REAL FX MODELING ENGINE READY")
print("🚀 Loading actual CBK exchange rate data...")
print("="*60)

🚀 Real FX Modeling Engine Initialized
💱 Loading actual CBK exchange rate data...
\n============================================================
💱 REAL FX MODELING ENGINE READY
🚀 Loading actual CBK exchange rate data...


In [6]:
# Load and Process Real FX Data - Clean Version
print("LOADING REAL CBK EXCHANGE RATE DATA")
print("="*50)

# Direct approach to load USD/KES exchange rate data
try:
    # Load the monthly exchange rate data
    fx_raw = pd.read_csv("../data/raw/Monthly exchange rate (end period).csv")
    print(f"Raw FX data loaded: {len(fx_raw)} rows")
    
    # The data structure: Row 0 has headers, data starts from row 1
    # Columns: Year, Month, USD Rate, Sterling, Euro, etc.
    fx_data = fx_raw.iloc[1:].copy()  # Skip header row
    
    # Rename columns for easier access
    fx_data.columns = ['Year', 'Month', 'USD_KES', 'Sterling', 'Euro'] + list(fx_data.columns[5:])
    
    # Convert to proper types
    fx_data['Year'] = pd.to_numeric(fx_data['Year'], errors='coerce')
    fx_data['Month'] = pd.to_numeric(fx_data['Month'], errors='coerce')
    fx_data['USD_KES'] = pd.to_numeric(fx_data['USD_KES'], errors='coerce')
    
    # Create date column
    fx_data['Date'] = pd.to_datetime(fx_data[['Year', 'Month']].assign(day=1), errors='coerce')
    
    # Clean data - remove rows with missing values
    fx_clean = fx_data.dropna(subset=['Date', 'USD_KES']).copy()
    fx_clean = fx_clean.sort_values('Date').reset_index(drop=True)
    
    print(f"Processed FX data: {len(fx_clean)} valid records")
    print(f"Date range: {fx_clean['Date'].min()} to {fx_clean['Date'].max()}")
    print(f"USD/KES range: {fx_clean['USD_KES'].min():.2f} - {fx_clean['USD_KES'].max():.2f}")
    
    # Add analysis columns
    fx_clean['Rate_Change'] = fx_clean['USD_KES'].diff()
    fx_clean['Rate_PCT_Change'] = fx_clean['USD_KES'].pct_change()
    fx_clean['Volatility_6M'] = fx_clean['Rate_PCT_Change'].rolling(window=6).std()
    
    # Create simple visualization
    import plotly.graph_objects as go
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=fx_clean['Date'],
        y=fx_clean['USD_KES'],
        mode='lines+markers',
        name='USD/KES Exchange Rate',
        line=dict(color='blue', width=2),
        marker=dict(size=3)
    ))
    
    fig.update_layout(
        title="Real CBK USD/KES Exchange Rate (1993-2023)",
        xaxis_title="Date",
        yaxis_title="USD/KES Rate",
        height=500
    )
    
    fig.show()
    
    # Display key insights
    current_rate = fx_clean['USD_KES'].iloc[-1]
    avg_rate = fx_clean['USD_KES'].mean()
    rate_change_1y = fx_clean['USD_KES'].iloc[-1] / fx_clean['USD_KES'].iloc[-12] - 1 if len(fx_clean) >= 12 else 0
    
    print(f"\\nKEY FX INSIGHTS:")
    print(f"   Current USD/KES Rate: {current_rate:.2f}")
    print(f"   Historical Average: {avg_rate:.2f}")
    print(f"   Annual Change: {rate_change_1y*100:.2f}%")
    print(f"   Data Points: {len(fx_clean)} monthly observations")
    print(f"   Volatility: {fx_clean['Rate_PCT_Change'].std()*100:.2f}%")
    
    # Store for further analysis
    processed_fx = fx_clean.copy()
    
except Exception as e:
    print(f"Error loading FX data: {str(e)}")
    processed_fx = None

LOADING REAL CBK EXCHANGE RATE DATA
Raw FX data loaded: 391 rows
Processed FX data: 390 valid records
Date range: 1993-01-01 00:00:00 to 2025-06-01 00:00:00
USD/KES range: 35.92 - 160.75


\nKEY FX INSIGHTS:
   Current USD/KES Rate: 129.23
   Historical Average: 85.31
   Annual Change: -0.53%
   Data Points: 390 monthly observations
   Volatility: 3.36%


In [ ]:
# Advanced FX Modeling & Prediction
print("\\n🧠 ADVANCED FX MODELING & PREDICTION")
print("="*50)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

class AdvancedFXPredictor:
    def __init__(self):
        self.models = {
            'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
            'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
            'Linear Regression': LinearRegression()
        }
        self.scaler = MinMaxScaler()
        self.best_model = None
        self.model_performance = {}
    
    def create_features(self, df):
        """Create advanced features for FX prediction"""
        features_df = df.copy()
        
        # Technical indicators
        features_df['ma_7'] = features_df['usd_kes_rate'].rolling(window=7).mean()
        features_df['ma_30'] = features_df['usd_kes_rate'].rolling(window=30).mean()
        features_df['ma_ratio'] = features_df['ma_7'] / features_df['ma_30']
        
        # Volatility features
        features_df['volatility_7'] = features_df['usd_kes_rate'].rolling(window=7).std()
        features_df['volatility_30'] = features_df['usd_kes_rate'].rolling(window=30).std()
        
        # Rate of change features
        features_df['roc_1'] = features_df['usd_kes_rate'].pct_change(1)
        features_df['roc_7'] = features_df['usd_kes_rate'].pct_change(7)
        features_df['roc_30'] = features_df['usd_kes_rate'].pct_change(30)
        
        # Lagged features
        for lag in [1, 2, 3, 7, 14]:
            features_df[f'lag_{lag}'] = features_df['usd_kes_rate'].shift(lag)
        
        # Time-based features
        features_df['month'] = features_df.index.month
        features_df['quarter'] = features_df.index.quarter
        features_df['year'] = features_df.index.year
        
        return features_df.dropna()
    
    def prepare_training_data(self, df, target_col='usd_kes_rate', test_size=0.2):
        """Prepare data for training"""
        feature_cols = [col for col in df.columns if col != target_col]
        
        # Split data
        split_idx = int(len(df) * (1 - test_size))
        train_df = df.iloc[:split_idx]
        test_df = df.iloc[split_idx:]
        
        X_train = train_df[feature_cols]
        y_train = train_df[target_col]
        X_test = test_df[feature_cols]
        y_test = test_df[target_col]
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        return X_train_scaled, X_test_scaled, y_train, y_test, feature_cols
    
    def train_models(self, X_train, X_test, y_train, y_test):
        """Train multiple models and evaluate performance"""
        results = {}
        
        for name, model in self.models.items():
            print(f"\\n🤖 Training {name}...")
            
            # Train model
            model.fit(X_train, y_train)
            
            # Make predictions
            y_pred = model.predict(X_test)
            
            # Calculate metrics
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            
            results[name] = {
                'model': model,
                'mae': mae,
                'mse': mse,
                'rmse': rmse,
                'r2': r2,
                'predictions': y_pred
            }
            
            print(f"   📊 MAE: {mae:.4f}")
            print(f"   📊 RMSE: {rmse:.4f}")
            print(f"   📊 R²: {r2:.4f}")
        
        # Find best model
        best_model_name = min(results.keys(), key=lambda x: results[x]['rmse'])
        self.best_model = results[best_model_name]['model']
        self.model_performance = results
        
        print(f"\\n🏆 Best Model: {best_model_name} (RMSE: {results[best_model_name]['rmse']:.4f})")
        
        return results
    
    def predict_future(self, last_features, periods=30):
        """Generate future predictions"""
        if self.best_model is None:
            raise ValueError("No trained model available")
        
        predictions = []
        current_features = last_features.copy()
        
        for _ in range(periods):
            # Scale features
            scaled_features = self.scaler.transform([current_features])
            
            # Make prediction
            pred = self.best_model.predict(scaled_features)[0]
            predictions.append(pred)
            
            # Update features for next prediction (simplified approach)
            current_features = np.roll(current_features, -1)
            current_features[-1] = pred
        
        return predictions

# Initialize predictor
predictor = AdvancedFXPredictor()

if 'processed_fx' in locals() and processed_fx is not None:
    print("\\n🔧 Creating advanced features...")
    features_df = predictor.create_features(processed_fx)
    
    print(f"   📊 Features created: {len(features_df.columns) - 1}")
    print(f"   📊 Training samples: {len(features_df)}")
    
    # Prepare training data
    print("\\n🎯 Preparing training data...")
    X_train, X_test, y_train, y_test, feature_cols = predictor.prepare_training_data(features_df)
    
    print(f"   📊 Training size: {len(X_train)}")
    print(f"   📊 Testing size: {len(X_test)}")
    
    # Train models
    print("\\n🚀 Training prediction models...")
    model_results = predictor.train_models(X_train, X_test, y_train, y_test)
    
else:
    print("❌ No processed FX data available for modeling")

In [ ]:
# Model Performance Visualization & Future Predictions
print("\\n📊 MODEL PERFORMANCE & FUTURE PREDICTIONS")
print("="*50)

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from datetime import datetime, timedelta

def create_model_comparison_chart(model_results, y_test):
    """Create comprehensive model comparison visualization"""
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Model Performance Comparison', 'Prediction vs Actual', 
                       'Residuals Analysis', 'Feature Importance'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Model performance metrics
    models = list(model_results.keys())
    rmse_values = [model_results[model]['rmse'] for model in models]
    r2_values = [model_results[model]['r2'] for model in models]
    
    fig.add_trace(
        go.Bar(name='RMSE', x=models, y=rmse_values, marker_color='lightcoral'),
        row=1, col=1
    )
    
    # Best model predictions vs actual
    best_model_name = min(models, key=lambda x: model_results[x]['rmse'])
    best_predictions = model_results[best_model_name]['predictions']
    
    fig.add_trace(
        go.Scatter(name='Actual', y=y_test.values, mode='lines', line=dict(color='blue')),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(name='Predicted', y=best_predictions, mode='lines', line=dict(color='red')),
        row=1, col=2
    )
    
    # Residuals
    residuals = y_test.values - best_predictions
    fig.add_trace(
        go.Scatter(x=best_predictions, y=residuals, mode='markers', 
                  name='Residuals', marker=dict(color='green')),
        row=2, col=1
    )
    
    # R² comparison
    fig.add_trace(
        go.Bar(name='R²', x=models, y=r2_values, marker_color='lightblue'),
        row=2, col=2
    )
    
    fig.update_layout(
        title="🎯 FX Model Performance Analysis",
        height=800,
        showlegend=True,
        title_font=dict(size=16, color='darkblue')
    )
    
    return fig

def create_future_predictions_chart(historical_data, predictions, periods=30):
    """Create future predictions visualization"""
    # Generate future dates
    last_date = historical_data.index[-1]
    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=periods, freq='D')
    
    fig = go.Figure()
    
    # Historical data
    fig.add_trace(
        go.Scatter(
            x=historical_data.index[-100:],  # Last 100 points
            y=historical_data['usd_kes_rate'].iloc[-100:],
            mode='lines',
            name='Historical USD/KES',
            line=dict(color='blue', width=2)
        )
    )
    
    # Future predictions
    fig.add_trace(
        go.Scatter(
            x=future_dates,
            y=predictions,
            mode='lines+markers',
            name='Predicted USD/KES',
            line=dict(color='red', width=2, dash='dash'),
            marker=dict(size=6)
        )
    )
    
    # Add vertical line at prediction start
    fig.add_vline(
        x=last_date,
        line_dash="dot",
        line_color="gray",
        annotation_text="Prediction Start"
    )
    
    fig.update_layout(
        title="🔮 USD/KES Exchange Rate Forecast (Next 30 Days)",
        xaxis_title="Date",
        yaxis_title="USD/KES Rate",
        height=600,
        showlegend=True,
        title_font=dict(size=16, color='darkblue')
    )
    
    return fig

# Generate visualizations if models are trained
if 'model_results' in locals() and model_results:
    print("\\n📊 Creating model comparison chart...")
    comparison_chart = create_model_comparison_chart(model_results, y_test)
    comparison_chart.show()
    
    # Generate future predictions
    print("\\n🔮 Generating future predictions...")
    try:
        # Get last features for prediction
        last_features = X_test[-1]  # Use last test sample as starting point
        future_predictions = predictor.predict_future(last_features, periods=30)
        
        # Create predictions chart
        predictions_chart = create_future_predictions_chart(processed_fx, future_predictions)
        predictions_chart.show()
        
        # Display prediction summary
        print(f"\\n🎯 FUTURE PREDICTIONS SUMMARY:")
        print(f"   🔮 30-Day Forecast Range: {min(future_predictions):.2f} - {max(future_predictions):.2f}")
        print(f"   📈 Predicted Direction: {'Depreciation' if future_predictions[-1] > future_predictions[0] else 'Appreciation'}")
        print(f"   💱 Current Rate: {processed_fx['usd_kes_rate'].iloc[-1]:.2f}")
        print(f"   🎯 30-Day Target: {future_predictions[-1]:.2f}")
        print(f"   📊 Expected Change: {((future_predictions[-1] / processed_fx['usd_kes_rate'].iloc[-1]) - 1) * 100:.2f}%")
        
        # Risk assessment
        volatility = np.std(future_predictions)
        print(f"\\n⚠️ RISK ASSESSMENT:")
        print(f"   📊 Prediction Volatility: {volatility:.4f}")
        print(f"   🎯 Confidence Level: {'High' if volatility < 2.0 else 'Medium' if volatility < 5.0 else 'Low'}")
        
    except Exception as e:
        print(f"❌ Error generating future predictions: {str(e)}")
        
else:
    print("❌ No trained models available for visualization")